In [6]:
import cv2
import numpy as np
from keras.preprocessing import image
from keras.models import load_model

In [7]:
# Load trained model
model = load_model("model/final_emotion_model.hdf5", compile=False)

In [8]:
# Cascadeclassifier is a classifier provided by OpenCV
# HaarCascade can be used to detect human face 
face_cascade = cv2.CascadeClassifier('model/haarcascade_frontalface_default.xml')

In [9]:
# VideoCapture, 0 represents laptop's camera, 1 represents USB camera, it can also input video path to recognize emotion in video
cap = cv2.VideoCapture(1)

In [10]:
# Call the camera when it is available
while True:
    # Read video or camera picture by frame
    # ret, frame is cap the two return values of the read (), where ret is a Boolean value. If the read frame is correct, it returns true.
    ret, frame = cap.read()
    if not ret:
        break
    # Transfer the image from RGB to gray space and convert it into gray image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Recognize the faces in the image and return the rectangular box vector group of all faces
    # To detect objects of different sizes, the length and width are gradually reduced in a certain proportion through the scalefactor parameter
    faces = face_cascade.detectMultiScale(gray, 1.32, 2)
    
    # Frame the face with a rectangle
    # Try except to avoid error
    try:
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            # Return the cropped face from the image
            roi_gray = gray[y:y+h, x:x+w]
            # Convert the image into an array.
            # Desired output image size
            roi_gray = cv2.resize(roi_gray, (48, 48))
            # The element type before conversion is integer and after is floating
            pixels = image.img_to_array(roi_gray)
            # Processing pixels
            pixels = np.expand_dims(pixels, axis = 0)
            pixels /= 255
           
            # Model prediction, input test set, output prediction results
            predictions = model.predict(pixels)
            # When several maximum values appear in a group, the index value of the first maximum value is returned.
            max_index = np.argmax(predictions[0])
            emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            # Returns the index value of the largest value in a numpy array.
            pred_emotions = emotions[max_index]
            # Print emotion and its confidence value
            cv2.putText(frame, pred_emotions, (int(x),int(y-50)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)       
            confidence = 'Confidence: {}'.format(str(np.round(np.max(predictions[0])*100,1))+ "%")
            cv2.putText(frame, confidence, (int(x),int(y-10)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    except:
        pass
    # Frame size
    resized_frame = cv2.resize(frame, (1100, 800))
    cv2.imshow('frame', resized_frame)
    # Press the 'z' key on the keyboard to close the program
    if cv2.waitKey(1) & 0xFF == ord('z'):
        break
cap.release() # Stop capturing video
cv2.destroyAllWindows() # Close the corresponding window